In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('chem.csv', delimiter=';')

/Users/jayarajbalagopal/IIITH/Semester2/SMAI/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,ChEMBL ID,Name,Synonyms,Type,Max Phase,Molecular Weight,Targets,Bioactivities,AlogP,Polar Surface Area,...,Structure Type,Inorganic Flag,Heavy Atoms,HBA (Lipinski),HBD (Lipinski),#RO5 Violations (Lipinski),Molecular Weight (Monoisotopic),Molecular Species,Molecular Formula,Smiles
0,CHEMBL166302,NaN,NaN,Small molecule,0,448.31,1.0,3.0,3.63,73.86,...,MOL,-1,28,6,1,0,447.0681,NEUTRAL,C21H22BrNO5,COc1cc2c(c(OC)c1OC)-c1ccc(Br)c(=O)cc1[C@@H](NC...
1,CHEMBL238119,NaN,NaN,Small molecule,0,321.46,3.0,3.0,6.18,44.02,...,MOL,-1,24,2,1,1,321.2093,NEUTRAL,C22H27NO,CCCCCCC(C)(C)c1ccc(-c2cccc(C#N)c2)c(O)c1
2,CHEMBL1598344,NaN,NaN,Small molecule,0,482.63,13.0,15.0,3.60,47.10,...,MOL,-1,36,6,0,0,482.2682,NEUTRAL,C30H34N4O2,O=C(c1cccc(C(=O)N2CCN(Cc3ccccc3)CC2)c1)N1CCN(C...
3,CHEMBL2139018,NaN,NaN,Small molecule,0,481.39,10.0,13.0,6.00,44.99,...,MOL,-1,31,5,0,1,480.1049,NEUTRAL,C25H25BrN2O3,COc1ccc(Cc2c(-c3ccc(OC)c(OC)c3)nc3c(C)cc(Br)cn...
4,CHEMBL1542218,NaN,NaN,Small molecule,0,373.52,10.0,11.0,3.78,66.89,...,MOL,-1,26,5,2,0,373.1824,NEUTRAL,C20H27N3O2S,CC1CCCC(NC(=O)CCCn2c(=S)[nH]c3ccccc3c2=O)C1C


In [19]:
df = df[df["Smiles"].str.contains("@")==False]

In [21]:
df = df[:100000]

In [22]:
df = df[['ChEMBL ID', 'Type', 'Molecular Weight', 'AlogP', 'Structure Type', 'Inorganic Flag', 'Heavy Atoms', 'Molecular Species', 'Smiles']]

In [24]:
df.head()

,ChEMBL ID,Type,Molecular Weight,AlogP,Structure Type,Inorganic Flag,Heavy Atoms,Molecular Species,Smiles
1,CHEMBL238119,Small molecule,321.46,6.18,MOL,-1,24,NEUTRAL,CCCCCCC(C)(C)c1ccc(-c2cccc(C#N)c2)c(O)c1
2,CHEMBL1598344,Small molecule,482.63,3.60,MOL,-1,36,NEUTRAL,O=C(c1cccc(C(=O)N2CCN(Cc3ccccc3)CC2)c1)N1CCN(C...
3,CHEMBL2139018,Small molecule,481.39,6.00,MOL,-1,31,NEUTRAL,COc1ccc(Cc2c(-c3ccc(OC)c(OC)c3)nc3c(C)cc(Br)cn...
4,CHEMBL1542218,Small molecule,373.52,3.78,MOL,-1,26,NEUTRAL,CC1CCCC(NC(=O)CCCn2c(=S)[nH]c3ccccc3c2=O)C1C
5,CHEMBL4755233,NaN,908.06,6.91,MOL,-1,65,BASE,CC(CCC1OC(c2ccc3c(c2)OCO3)OOC1(C)C)CCn1cc(COCC...


In [25]:
df.drop(df.loc[(df['Heavy Atoms']=='None')].index, inplace=True)
df.dropna(inplace=True)

In [26]:
df = df.loc[(df['Inorganic Flag'].astype(int) == -1) & (df['Heavy Atoms'].astype(int) >= 3) & (df['Molecular Weight'].astype(int) >= 12) & (df['Molecular Weight'].astype(int) <= 600)]
df = df.loc[(df['AlogP'].astype(float) >= -7) & (df['AlogP'].astype(float) <= 5)]
df = df.loc[(df['Structure Type'] == 'MOL')]

In [27]:
df = df[['ChEMBL ID', 'Smiles']]

In [30]:
df.to_csv('smiles.csv', sep='\t', encoding='utf-8')

In [28]:
!pip install svglib
!pip install reportlab

You should consider upgrading via the '/Users/jayarajbalagopal/IIITH/Semester2/SMAI/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jayarajbalagopal/IIITH/Semester2/SMAI/env/bin/python -m pip install --upgrade pip' command.


In [37]:
import os
import urllib
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from tqdm import tqdm

failed = []
for i in tqdm(range(len(df))):
    try:
        urllib.request.urlretrieve("https://www.ebi.ac.uk/chembl/api/data/image/{}".format(df.iloc[i]['ChEMBL ID']), "svg_images/{}.svg".format(df.iloc[i]['ChEMBL ID']))
#         drawing = svg2rlg('images/{}.svg'.format(df.iloc[i]['ChEMBL ID']))
#         renderPM.drawToFile(drawing, 'images/{}.png'.format(df.iloc[i]['ChEMBL ID']), fmt='PNG')
#         os.remove('images/{}.svg'.format(df.iloc[i]['ChEMBL ID']))
    except Exception as e:
        print(df.iloc[i]['ChEMBL ID'])
        failed.append(i)

  0%|                                     | 20/75233 [00:22<23:31:26,  1.13s/it]


KeyboardInterrupt: 

In [ ]:
import os
import urllib
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from tqdm import tqdm

failed = []
for i in tqdm(range(len(df))):
    try:
        urllib.request.urlretrieve("https://www.ebi.ac.uk/chembl/api/data/image/{}".format(df.iloc[i]['ChEMBL ID']), "images/{}.svg".format(df.iloc[i]['ChEMBL ID']))
        drawing = svg2rlg('images/{}.svg'.format(df.iloc[i]['ChEMBL ID']))
        renderPM.drawToFile(drawing, 'images/{}.png'.format(df.iloc[i]['ChEMBL ID']), fmt='PNG')
        os.remove('images/{}.svg'.format(df.iloc[i]['ChEMBL ID']))
    except Exception as e:
        print(df.iloc[i]['ChEMBL ID'])
        failed.append(i)

In [141]:
failed

[]